In [1]:
from constants import *

In [2]:
import os
os.environ['PATH'] = "/usr/local/opt/php@7.4/bin:" + os.environ['PATH']

In [3]:
import os
from dotenv import load_dotenv
from pathlib import Path
rel_path = Path('../../../Python') / '.env'
load_dotenv(rel_path)
key_mnemonic = os.getenv("MNEMONIC")

In [4]:
key_mnemonic

'budget similar slight coil wage cupboard infant joke expand among anxiety trim'

In [5]:
private_key = os.getenv("MYCRYPTO_PRIVATE_KEY")
from web3 import Web3
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
from eth_account import Account
account_one = Account.from_key(private_key)

In [6]:
print(account_one.address)

0x53F761Ed45A9f105D27243604fCEA98b2F6E1095


In [7]:
import subprocess
import json
def derive_wallets(mnemonic, coin, numderive):
#     command = 'php hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --cols=path,address,privkey,pubkey --format=json'
    command = 'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="' + mnemonic + '" --cols=path,address,privkey,pubkey --coin="' + coin + '" --numderive="' + str(numderive) + '" --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    keys = json.loads(output)
    return keys

In [8]:
eth_keys = derive_wallets(key_mnemonic, ETH, 1)
eth_keys

{'path': "m/44'/60'/0'/0/0",
 'address': '0x48f0152d5141cAD0dB3f859f842479F15BD5C945',
 'privkey': '0xe31a77d4324d7bb8f631f85db737395562cc8a48ad8285dfee1ca3798b0b8cb2',
 'pubkey': '0393e87ec29dc33af2deb6a78bf1188455eae2bcc1456700dff4359384f4a273b8'}

In [9]:
btc_keys = derive_wallets(key_mnemonic, BTC, 3)
btc_keys

[{'path': "m/44'/0'/0'/0/0",
  'address': '17tzQ4MjGoXewJaqZqWD31e7JhXhLo85SK',
  'privkey': 'L5jmeNgCzPhaEKSWq2YuhGvc3x5EthExV9HPcgZzHBs7M8A471a9',
  'pubkey': '03b164aadc27eae27cb89d2e8b5a4221f884e6fe5a25654471a1660a435fe0f034'},
 {'path': "m/44'/0'/0'/0/1",
  'address': '1NJJihKtxfYdVtcXxstQhYHbGEzozCDjjZ',
  'privkey': 'L26QaFtfxBWK291wPNBNm9MCmQbQoj7M6yXPyiq4e7dfaM3D8T8j',
  'pubkey': '0214d6bdd2fcd6c5745c1985fe17707b8b0d16ae99673a5c789727abdcd288541c'},
 {'path': "m/44'/0'/0'/0/2",
  'address': '1Gkcwbu3eUaQbTQjjpm1hyKZYevLFtRnFC',
  'privkey': 'KyoKMyh93pjQLTdwxA24rfCo7r9RagCS5fA3YPStZovGsVPdncGo',
  'pubkey': '0399601cdae06ce05c223b7e01eb895e32395985f9f30d0445d41c775c0b0a9929'}]

In [10]:
btctest_keys = derive_wallets(key_mnemonic, BTCTEST, 1)
btctest_keys

{'path': "m/44'/1'/0'/0/0",
 'address': 'mnEGbDqciJdaQVUNuAkR6fYe8NxhNtLtq9',
 'privkey': 'cPFE55yDqGfteVqNJi9U2NcfRqwvE4uK3TdfRk1ExnMzp9TZZnci',
 'pubkey': '0200892557f112882274dd3e17d6aecc971618cb5a7c4b9f448a7bf06f56f032c2'}

In [11]:
from bit import PrivateKeyTestnet
def priv_key_to_account(coin, priv_key):
    if coin.lower() == 'eth':
        return Account.privateKeyToAccount(priv_key)
    else:
        return PrivateKeyTestnet(priv_key)

In [12]:
eth_account = priv_key_to_account(ETH, eth_keys['privkey'])
eth_account

In [13]:
btctest_account = priv_key_to_account(BTCTEST, btctest_keys['privkey'])
btctest_account

<PrivateKeyTestnet: mnEGbDqciJdaQVUNuAkR6fYe8NxhNtLtq9>

In [14]:
btc_account = priv_key_to_account(BTC, btc_keys[0]['privkey'])
btc_account

<PrivateKeyTestnet: mnQwh7Si5pxuiR4THQUarvrSAh8QKS1JFE>

In [15]:
def create_tx(coin, account, recipient, amount):
    if coin.lower() == 'eth':
        print('ETH')
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )   
        return {
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
#             "chainID": w3.eth.chainId,
        }
    else:
        print('not ETH')
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [16]:
eth_create = create_tx(ETH, eth_account, account_one.address, 10)
eth_create

ETH


{'from': '0x48f0152d5141cAD0dB3f859f842479F15BD5C945',
 'to': '0x53F761Ed45A9f105D27243604fCEA98b2F6E1095',
 'value': 10,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 8}

In [17]:
btc_create = create_tx(BTC, btc_account, btc_keys[1]['address'], 10)
btc_create

not ETH


ValueError: Transactions must have at least one unspent.

In [ ]:
btctest_create = create_tx(BTCTEST, btctest_account, btctest_keys['address'], 10)
btctest_create

In [18]:
from bit.network import NetworkAPI
def send_tx(coin, account, recipient, amount):
    if coin.lower() == 'eth':
        print('ETH')
        raw_tx = create_tx(coin, account, recipient, amount)
        signed_tx = account.sign_transaction(raw_tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result)
    else:
        print('not ETH')
        raw_tx = create_tx(coin, account, recipient, amount)
        signed_tx = account.sign_transaction(raw_tx)
        result = NetworkAPI.broadcast_tx_testnet(signed_tx)
        print(result)    
    return result

In [19]:
eth_send = send_tx(ETH, eth_account, account_one.address, 10)
eth_send

ETH
ETH
b'P\x1bK]\x1a\xfb\x05\xd9\xd1(\x13INq\xd5\x9a\xe3\xfe\xecDc\x97\x80\xc8\xbb\x8b\x0c|\xe1\xdb\xb8\xbb'


HexBytes('0x501b4b5d1afb05d9d12813494e71d59ae3feec44639780c8bb8b0c7ce1dbb8bb')

In [ ]:
btc_send = send_tx(BTC, btc_account, btc_keys[1]['address'], 0.000000001)
btc_send

In [20]:
btctest_send = send_tx(BTCTEST, btctest_account, btctest_keys['address'], 0.0000000000000000000000000001)
btctest_send

not ETH
not ETH


InsufficientFunds: Balance 99408 is less than 111248 (including fee).